# AUTO$^2$ and AUTO-Demos : *pp3* - Periodic families in a 3D predator-prey system with harvesting.

**This is an example on how to use AUTO$^2$ to explore the AUTO Demos *pp3* bifurcations and solutions.**

This example is concerned with a 3D predator-prey system with harvesting (Doedel (1984)).

The equations of the model are given by:

$$
\dot u_1 = u_1 (1-u_1) - p_4 u_1 u_2
$$
$$
\dot u_2 = -p_2 u_2 + p_4 u_1 u_2 - p_5 u_2 u_3 - p_1 (1 - e^{-p_6 u_2})
$$
$$
\dot u_3 = - p_3 u_3 + p_5 u_2 u_3
$$

The free parameter in the current example notebook is $p_1$ . Other parameters are fixed, namely $p_2 = $0.25, $p_3 = $0.5,
$p_4 = $4, $p_5 = $3, and $p_6 = $5.

We are thus going to find the fixed points and periodic orbits of this system and continue them by varying $p_1$.

#### References

* Doedel, E. (1984). The computer-aided bifurcation analysis of predator-prey models. Journal of Mathematical Biology, 20(1), 1-14. [doi:10.1007/BF00275858](https://doi.org/10.1007/BF00275858).

## Code

First we set the Python path if needed:

In [ ]:
import sys, os

In [ ]:
sys.path.extend([os.path.abspath('../../../')])

And load the needed libraries, including AUTO$^2$:

In [ ]:
import numpy as np
from numba import njit
from scipy.optimize import root
from scipy.integrate import solve_ivp

In [ ]:
from auto2.diagrams.bifurcations import BifurcationDiagram

Creating the model equations

In [ ]:
@njit
def pp3(X, p):
    u1 = X[0]
    u2 = X[1]
    u3 = X[2]
    p1 = p[0]
    p2 = p[1]
    p3 = p[2]
    p4 = p[3]
    p5 = p[4]
    p6 = p[5]
    F = np.zeros(3)
    F[0] = u1 * (1 - u1) - p4 * u1 * u2
    F[1] = - p2 * u2 + p4 * u1 * u2 - p5 * u2 * u3 - p1 * (1 - np.exp(-p6 * u2))
    F[2] = - p3 * u3 + p5 * u2 * u3
    return F

@njit
def pp3t(t, X, p):
    return pp3(X, p)

and define a set of standard parameters:

In [ ]:
params = {
    'p': (0.375, 0.25, 0.5, 4., 3., 5.),
    }

For reference later, we can compute a long trajectory on the attractor of this model:

In [ ]:
#first a transient
ic = np.zeros(3) + 0.01
transient = solve_ivp(pp3t, (0., 10000.), ic, args=tuple(params.values()))

In [ ]:
# then the trajectory itself
ic = transient['y'][:, -1]
trajectory = solve_ivp(pp3t, (0., 100000.), ic, args=tuple(params.values()))

Finding all the fixed points of the system for $p_1 = 0$ :

In [ ]:
params = {
    'p': (0.0, 0.25, 0.5, 4., 3., 5.),
    }

In [ ]:
nsearch = 1000

# Start on random initial conditions
ic = 2 * (np.random.rand(nsearch, 3) - 0.5) * 10.

eps = 1.e-6
fixed_points = dict()

sol_idx = 1
for i in range(nsearch):
    sol = root(pp3, ic[i, :], args=tuple(params.values()))
    if sol.success:
        for idx in fixed_points:
            if np.linalg.norm(fixed_points[idx] - sol.x) < eps:
                break
        else:
            fixed_points[sol_idx] = sol.x
            sol_idx+=1


In [ ]:
fixed_points

We have now the list of fixed points `fixed_points` and parameters dictionnary `params` that AUTO$^2$ will have to continue

In [ ]:
initial_points = list()

for p in fixed_points:
    initial_points.append({'parameters': {'p'+str(i+1): params['p'][i] for i in range(len(params['p']))}, 'initial_data': fixed_points[p]})


In [ ]:
initial_points

and thus we are now ready to compute the diagram of fixed points as a function of $p_1$. Note that we specify that the bifurcation diagram object must load the ̀`pp3.f90` and `c.pp3` files where the model equations and continuation parameters have been written:

In [ ]:
b = BifurcationDiagram('pp3')

b.compute_fixed_points_diagram(initial_points,extra_comparison_parameters=['u1', 'u2', 'u3'], comparison_tol=[1.e-3] * 4,
                               ICP=['p1'], NMX=500, UZSTOP={'p1':[1., -1.]}, UZR={'p1': [-0.25,0.,0.25,0.375,0.5,0.75]})

We can now plot the result as functions of $p_1$ and $L^2$ norm :

In [ ]:
b.plot_fixed_points_diagram((0,1));

or in 3D as functions of $p_1$, $L^2$ norm and $u_1$ :

In [ ]:
b.plot_fixed_points_diagram_3D();

We see that at many Hopf bifurcations were found.

We can continue periodic orbits out of these Hopf bifurcations : 

In [ ]:
b.compute_periodic_orbits_diagram(3, extra_comparison_parameters=['u1', 'u2', 'u3'], max_number_bp=None, comparison_tol=[1.e-3] * 4,
                                  ICP=['p1'], NMX=500)


and plot the results on a bifurcation diagram:

In [ ]:
ax = b.plot_fixed_points_diagram()
b.plot_periodic_orbits_diagram(ax=ax, cmap='gist_ncar');
ax.set_xlim((-0.25,0.75))
ax.set_ylim((-0.1,1.))

In [ ]:
ax = b.plot_fixed_points_diagram_3D()
b.plot_periodic_orbits_diagram_3D((0,1,4),ax=ax, cmap='gist_ncar');


We can also plot both the bifurcation diagram and the solutions for a given value of $p_1$:

In [ ]:
axs = b.plot_diagram_and_solutions(0.375, solutions_variables=(0, 1), fixed_points_diagram_kwargs={'legend': True}, 
                             periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar'})
axs[0].set_xlim((-0.25,0.75))
axs[0].set_ylim((-0.1,1.))
axs[1].set_ylim((-0.1,0.5))
axs[1].set_xlim((-0.1,1.1))

We can also plot the result in 3 dimensions to get a better view:

In [ ]:
b.plot_diagram_in_3D_and_solutions_in_3D(0.375, solutions_variables=(0, 1, 2), fixed_points_diagram_kwargs={}, 
                                         periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar'});

Finally, it is not hard to also plot the dynamics on the attractor (represented by the long trajectory computed beforehand) on top of the solutions to see their relevance:

In [ ]:
axs = b.plot_diagram_and_solutions(0.375, solutions_variables=(0, 1), fixed_points_diagram_kwargs={'legend': True}, 
                             periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar'})
axs[1].plot(trajectory['y'][0], trajectory['y'][1], marker='o', ms=0.7, ls='', color='darkgray')
axs[0].set_xlim((-0.25,0.75))
axs[0].set_ylim((-0.1,1.))
axs[1].set_ylim((-0.1,0.5))
axs[1].set_xlim((-0.1,1.1))

The dynamics looks irregular and seems to be contained between two of the found periodic orbits : 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(trajectory['y'][1][:1000], marker='o', ms=1.7, ls='-', color='darkgray')
plt.ylabel('$u_2$')
plt.xlabel('time');

From here, you can restart this notebook with other parameters values.